In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.metrics import matthews_corrcoef, confusion_matrix

## Carga de bandas para obtener máscaras originales y entrenar el modelo de Regresión Logística

In [ ]:
bandas = {'Bands_10':['B4 Red [665 nm]', 'B3 Green [560 nm]', 'B2 Blue [490 nm]', 'B8 Near infrared [842 nm]'],
          'Bands_20':['B8A Vegetation classification [865 nm]', 'B11 Snow ice cloud discrimination [1610 nm]', 
                         'B12 Snow ice cloud discrimination [2190 nm]']}

array_bandas = {}

indices = {}

def CargaBandas(archivo):
    
    for resolucion in bandas:
        
        netcdf = Dataset(os.path.join('/mnt/Oliver/La_Robleda', archivo, '{}.nc'.format(resolucion)), 'r')
        
        for banda in bandas[resolucion]:
            
            if resolucion == 'Bands_10':
            
                array = netcdf.variables[banda][:4000, 8000:]
                
                     
            elif resolucion == 'Bands_20':
                
                array = netcdf.variables[banda][:2000, 4000:]
            
            
            array_bandas[banda.split(' ')[0]] = array

            
    return array_bandas


def CargaIndices(archivo):
    
    
    ndwi = (array_bandas['B3'] - array_bandas['B8'])/(array_bandas['B3'] + array_bandas['B8'])
    ndwi_res = cv2.resize(ndwi, (1490,2000), interpolation = cv2.INTER_CUBIC)
    indices['ndwi'] = ndwi_res
    ndwi_res[ndwi_res >= 0.2] = np.nan
    ndwi_res = np.ma.masked_where(condition=np.isnan(ndwi_res), a=ndwi_res)
      
    B11 = cv2.resize(array_bandas['B11'], (2980,4000), interpolation=cv2.INTER_CUBIC)
    ndsi = (array_bandas['B3'] - B11)/(array_bandas['B3'] + B11)
    indices['ndsi'] = ndsi
    ndsi = cv2.resize(ndsi, (1490,2000), interpolation = cv2.INTER_CUBIC)

    B4 = cv2.resize(array_bandas['B4'], (1490,2000), interpolation = cv2.INTER_CUBIC)
    mask_potential_cloud = (B4 > 0.07) * (B4 < 0.25) # are passed to Step 1.b
    mask_cloud = B4 > 0.25 # Clouds , are passed to step2
    mask_cloud = mask_cloud + (mask_potential_cloud * (ndsi > -0.16))
    
       
    nbr = (array_bandas['B8A'] - array_bandas['B12'])/(array_bandas['B8A'] + array_bandas['B12'])
    nbr[ndwi_res.mask] = 1
    nbr[mask_cloud] = 1
    indices['nbr'] = nbr
    
    ndvi = (array_bandas['B8'] - array_bandas['B4'])/(array_bandas['B8'] + array_bandas['B4'])
    ndvi_res = cv2.resize(ndvi, (1490,2000), interpolation = cv2.INTER_CUBIC)
    ndvi_res[ndwi_res.mask] = 1
    ndvi_res[mask_cloud] = 1
    indices['ndvi'] = ndvi_res
    
    
    return indices


#archivo = 'S2A_MSIL2A_20170813T112121_N0205_R037_T29TPG_20170813T112433'

array_bandas = CargaBandas(archivo)

indices = CargaIndices(archivo)

In [ ]:
nbr_mask = indices['nbr']
nbr_mask[nbr_mask >= -0.2] = 1
nbr_mask[nbr_mask <= -0.2] = 0

ndvi_mask = indices['ndvi']
ndvi_mask[ndvi_mask <= 0] = 0
ndvi_mask[ndvi_mask > 0] = 1

In [ ]:
mascara = nbr_mask
indice = indices['nbr']

img =  np.zeros((array_bandas['B8A'].shape[0], array_bandas['B8A'].shape[1], 2))
img[:,:,0] = array_bandas['B8A']
img[:,:,1] = array_bandas['B12']
    
new_shape = (img.shape[0] * img.shape[1], img.shape[2])
X = img[:, :, :2].reshape(new_shape)
y_f = mascara.flatten()
    
X_train, X_test, y_train, y_test = train_test_split(X, y_f, test_size=0.8, random_state=42)
    

model=LogisticRegression(random_state=42).fit(X_train, y_train)


y_pred = model.predict(X_test)
print('Accuracy of test: {}'.format(accuracy_score(y_test, y_pred)))
    
mask_pred = model.predict(X)
print('Accuracy of complete mask: {}'.format(accuracy_score(y_f, mask_pred)))
mask_pred = mask_pred.reshape(indice.shape)

## Carga de bandas para los archivos de las nuevas fechas

In [ ]:
bandas = {'Bands_10':['B4 Red [665 nm]', 'B3 Green [560 nm]', 'B2 Blue [490 nm]', 'B8 Near infrared [842 nm]'],
          'Bands_20':['B8A Vegetation classification [865 nm]', 'B11 Snow ice cloud discrimination [1610 nm]', 
                         'B12 Snow ice cloud discrimination [2190 nm]']}

array_bandas = {}

indices = {}

def CargaBandas(archivo):
    
    for resolucion in bandas:
        
        netcdf = Dataset(os.path.join('/mnt/Oliver/La_Robleda', archivo, '{}.nc'.format(resolucion)), 'r')
        
        for banda in bandas[resolucion]:
            
            if resolucion == 'Bands_10':
            
                array = netcdf.variables[banda][:4000, 8000:]
                
                     
            elif resolucion == 'Bands_20':
                
                array = netcdf.variables[banda][:2000, 4000:]
            
            
            array_bandas[banda.split(' ')[0]] = array

            
    return array_bandas


def CargaIndices(archivo):
    
    
    ndwi = (array_bandas['B3'] - array_bandas['B8'])/(array_bandas['B3'] + array_bandas['B8'])
    ndwi_res = cv2.resize(ndwi, (1490,2000), interpolation = cv2.INTER_CUBIC)
    indices['ndwi'] = ndwi_res
    ndwi_res[ndwi_res >= 0.2] = np.nan
    ndwi_res = np.ma.masked_where(condition=np.isnan(ndwi_res), a=ndwi_res)
      
    B11 = cv2.resize(array_bandas['B11'], (2980,4000), interpolation=cv2.INTER_CUBIC)
    ndsi = (array_bandas['B3'] - B11)/(array_bandas['B3'] + B11)
    indices['ndsi'] = ndsi
    ndsi = cv2.resize(ndsi, (1490,2000), interpolation = cv2.INTER_CUBIC)

    B4 = cv2.resize(array_bandas['B4'], (1490,2000), interpolation = cv2.INTER_CUBIC)
    mask_potential_cloud = (B4 > 0.07) * (B4 < 0.25) # are passed to Step 1.b
    mask_cloud = B4 > 0.25 # Clouds , are passed to step2
    mask_cloud = mask_cloud + (mask_potential_cloud * (ndsi > -0.16))
    
       
    nbr = (array_bandas['B8A'] - array_bandas['B12'])/(array_bandas['B8A'] + array_bandas['B12'])
    nbr[ndwi_res.mask] = 1
    nbr[mask_cloud] = 1
    indices['nbr'] = nbr
    
    ndvi = (array_bandas['B8'] - array_bandas['B4'])/(array_bandas['B8'] + array_bandas['B4'])
    ndvi_res = cv2.resize(ndvi, (1490,2000), interpolation = cv2.INTER_CUBIC)
    ndvi_res[ndwi_res.mask] = 1
    ndvi_res[mask_cloud] = 1
    indices['ndvi'] = ndvi_res
    
    
    return indices

archivo = 'S2A_MSIL2A_20171012T112111_N0205_R037_T29TPG_20171012T112713'
#archivo = 'S2A_MSIL2A_20180619T112111_N0208_R037_T29TPG_20180619T142735'
#archivo = 'S2A_MSIL2A_20190214T112151_N0211_R037_T29TPG_20190214T121343'
#archivo = 'S2A_MSIL2A_20200219T112111_N0214_R037_T29TPG_20200219T124126'


array_bandas = CargaBandas(archivo)

indices = CargaIndices(archivo)

## Aplicación del modelo ya entrenado

In [ ]:
indice = indices['nbr']
mascara = nbr_mask

img =  np.zeros((array_bandas['B8A'].shape[0], array_bandas['B8A'].shape[1], 2))
img[:,:,0] = array_bandas['B8A']
img[:,:,1] = array_bandas['B12']
    
new_shape = (img.shape[0] * img.shape[1], img.shape[2])
X = img[:, :, :2].reshape(new_shape)
y_f = mascara.flatten()
    
mask_pred = model.predict(X)
print('Accuracy of complete mask: {}'.format(accuracy_score(y_f, mask_pred)))
mask_pred = mask_pred.reshape(indice.shape)

## Creación y aplicación de las máscaras para obtener la imagen final

In [ ]:
nbr = indices['nbr']
mask = mask_pred.astype(int)

m = np.ma.masked_where(nbr_mask ==1, nbr)
m = np.ma.masked_where(mask==0, m)

plt.imshow(m, origin='lower', cmap='RdYlGn')